<a href="https://colab.research.google.com/github/ndvp39/CloudComputing-tirgul/blob/main/Project/searchEngine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
!pip install requests beautifulsoup4
!pip install firebase
!pip install firebase-admin

In [37]:
from google.colab import drive, output
from IPython.display import HTML, Javascript
from IPython.display import clear_output
from firebase_admin import credentials
from firebase_admin import firestore
from nltk.chat.util import Chat, reflections
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup
import firebase_admin
import IPython
import random
import json
import os
import re
import requests
import nltk

nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

Github connection

In [55]:
# Specify the directory path
directory = '/content/CloudComputing-tirgul'

# Check if the directory exists
if os.path.exists(directory):
    # If the directory exists, remove it recursively
    !rm -r "$directory"

# Change the current directory to /content
os.chdir('/content')

# Clone the GitHub repository
!git clone "https://github.com/ndvp39/CloudComputing-tirgul"

# drive.mount('/content/drive')

Cloning into 'CloudComputing-tirgul'...
remote: Enumerating objects: 4370, done.
remote: Counting objects: 100% (1111/1111), done.
remote: Compressing objects: 100% (514/514), done.
remote: Total 4370 (delta 764), reused 862 (delta 564), pack-reused 3259
Receiving objects: 100% (4370/4370), 20.32 MiB | 17.67 MiB/s, done.
Resolving deltas: 100% (1758/1758), done.


Index:

In [57]:
# Run the index.ipynb notebook
%cd /content/CloudComputing-tirgul/Project/Index/
%run index.ipynb

/content/CloudComputing-tirgul/Project/Index


Get index from db

In [40]:
# GET INDEX FROM DB
index = get_index_from_db()

ChatBot

In [51]:
# Run the chatbot.ipynb notebook
%cd /content/CloudComputing-tirgul/Project/Chatbot/
%run chatbot.ipynb

/content/CloudComputing-tirgul/Project/Chatbot
/content/CloudComputing-tirgul/Project/Index


In [52]:
# Retrieve chatbot patterns from the database
db_collection_name = "ChatBotPatterns"
chatbot_patterns = get_patterns_from_db(db_collection_name)

# Create a new chatbot instance using the retrieved patterns and reflections
chatbot = Chat(chatbot_patterns, reflections)


SearchEngine

In [43]:
# Load and display a web page.
# Display HTML with optional CSS and JavaScript
# Supports two types of HTML docs - 1. combined with css and js | 2. 3 separate files
def switchToScreen(css_filename_, html_filename_, js_filename_):
    clear_output(wait=True) # Clear the current output to display the new content

    if css_filename_ is None and js_filename_ is None:
        html_response = requests.get(html_filename_)
        html_content_ = html_response.text
        display(HTML(html_content_))
    else:
        css_response = requests.get(css_filename_)
        css_content_ = css_response.text
        display(HTML(f'<style>{css_content_}</style>'))

        html_response = requests.get(html_filename_)
        html_content_ = html_response.text
        display(HTML(html_content_))

        js_response = requests.get(js_filename_)
        js_content_ = js_response.text
        display(Javascript(js_content_))

In [44]:
# Extract title from the HTML content of a given link.
def get_title_from_link(link):
    try:
        # Make an HTTP request
        response = requests.get(link)

        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Parse the HTML content
            soup = BeautifulSoup(response.text, 'html.parser')

            # Find the <h1> tag
            h1_tag = soup.find('h1')
            if h1_tag:
                return h1_tag.get_text()
            else:
                # If <h1> tag not found, try <h2> tag
                h2_tag = soup.find('h2')
                if h2_tag:
                    return h2_tag.get_text()
        else:
            # Return error message if request fails
            return f"Failed to fetch the page. Status code: {response.status_code}"

    except Exception as e:
        # Return error message if any exception occurs during extraction
        return f"An error occurred: {e}"

In [45]:
# Check for duplicate titles in a list and append index if found.
def check_title_list(lst):
  seen = {}
  result = []
  for word in lst:
    if word not in seen:
      seen[word] = 1
      result.append(word)
    else:
      seen[word] += 1
      result.append(f"{word}_{seen[word] - 1}")
  return result


# Retrieve titles, links, and query count for a given search string.
def get_titles_links_of_search(search_string):
  query_counter = 0
  titles = list()
  links = set()
  search_string = search_string.split()
  # trimming and steaming algorithem to search terms exactly as they are in the DB
  search_string = remove_stop_words(search_string)
  search_string = trim_words(search_string)
  search_string = apply_stemming(search_string)
  for word in index.keys():
    if word in search_string:
      for i in range(min(10, len(index[word]))): # 10 results max
        curr_link = getLinkFromString(word, i)
        query_counter += index[word][curr_link]['counter']
        if curr_link not in links:
          titles.append(get_title_from_link(curr_link))
        links.add(curr_link)
  titles = check_title_list(titles)
  return titles, links, query_counter

In [46]:
# Process the received search string and display search results in the GUI
def receive_search_string(search_string):

  # If search string is empty, return
  if search_string.strip() == "":
    return

  # Store original search string before modification
  original_search_string = search_string

  search_string = search_string.lower()

  # Switch to the results screen
  html_filename = f"https://raw.githubusercontent.com/ndvp39/CloudComputing-tirgul/main/Project/GUI/results.html"
  js_filename = f"https://raw.githubusercontent.com/ndvp39/CloudComputing-tirgul/main/Project/GUI/script.js"
  switchToScreen(css_filename, html_filename, js_filename)

  # JavaScript code to set the search bar text
  js_code = f"""
    <script>
      document.getElementById("searchbartext").value = "{original_search_string}";
    </script>
    """

  # Retrieve search results
  titles, links, times_appeared = get_titles_links_of_search(search_string)

  # Convert Python arrays to JavaScript arrays
  titles_js_array = ', '.join(f'"{title}"' for title in titles)
  links_js_array = ', '.join(f'"{link}"' for link in links)
  times_appeared_str = str(times_appeared)

  # JavaScript code to render search results
  js_code_2 = f"""
    <script>
      renderResults_title_link([ {titles_js_array} ], [ {links_js_array} ], [{times_appeared_str}]);
    </script>
  """

  display(HTML(js_code))
  display(HTML(js_code_2))

  # Update statistics if search results are available
  if titles:
    update_or_add_statistic(search_string)

# Register the function to be called from JavaScript
output.register_callback('notebook.receive_search_string', receive_search_string)

# Switch to the home page of the GUI.
def back_to_home_page():
  html_filename = f"https://raw.githubusercontent.com/ndvp39/CloudComputing-tirgul/main/Project/GUI/index.html"
  switchToScreen(css_filename, html_filename, js_filename)

# Register the function to be called from JavaScript
output.register_callback('notebook.back_to_home_page', back_to_home_page)


In [59]:
# Switch to the chatbot page of the GUI
def open_chatbot_page():
  html_filename = f"https://raw.githubusercontent.com/ndvp39/CloudComputing-tirgul/main/Project/GUI/chatbot.html"
  switchToScreen(css_filename, html_filename, js_filename)

# Register the function to be called from JavaScript
output.register_callback('notebook.open_chatbot_page', open_chatbot_page)

# Receive a question from the chatbot interface and provide a response
def receive_question_chatbot(question):
  empty_string = False

  if question == "":
    empty_string = True

  # Get response from the chatbot
  response = chatbot.respond(question)

  if response is None:
    response = "I'm unable to generate a response to that question. Is there anything else I can assist you with?"

  if empty_string:
    response = "It seems like your message is blank. How can I assist you today? If you have any questions or need assistance, feel free to ask!"

  if "May 6-9, 2024" in response or "An error occurred" in response:
    response = "An error has occurred. If the issue persists, please retry in a few seconds."
  # Generate JavaScript code to display the chatbot response
  js_code = f"""
      <script>
        ChatbotResponse("{response}");
      </script>
    """

  display(HTML(js_code))

# Register the function to be called from JavaScript
output.register_callback('notebook.receive_question_chatbot', receive_question_chatbot)

In [48]:
# Switch to the indexeditor page
def open_indexeditor_page():
  html_filename = "https://raw.githubusercontent.com/ndvp39/CloudComputing-tirgul/main/Project/GUI/admin.html"
  switchToScreen(None, html_filename, None)

# Register the function to be called from JavaScript
output.register_callback('notebook.open_indexeditor_page', open_indexeditor_page)

# Switch to the statistics page
def open_statistics_screen():
  html_filename = "https://raw.githubusercontent.com/ndvp39/CloudComputing-tirgul/main/Project/GUI/statistics.html"
  switchToScreen(None, html_filename, None)

# Register the function to be called from JavaScript
output.register_callback('notebook.open_statistics_screen', open_statistics_screen)

# Switch to the admin page
def back_to_admin():
  html_filename = "https://raw.githubusercontent.com/ndvp39/CloudComputing-tirgul/main/Project/GUI/admin.html"
  switchToScreen(None, html_filename, None)

# Register the function to be called from JavaScript
output.register_callback('notebook.back_to_admin', back_to_admin)


Administration:

In [49]:
# Run the administration.ipynb notebook
%cd /content/CloudComputing-tirgul/Project/Administration/
%run administration.ipynb

/content/CloudComputing-tirgul/Project/Administration
/content/CloudComputing-tirgul/Project/Index


Loading the main page

In [58]:
css_filename = f"https://raw.githubusercontent.com/ndvp39/CloudComputing-tirgul/main/Project/GUI/style.css"
js_filename = f"https://raw.githubusercontent.com/ndvp39/CloudComputing-tirgul/main/Project/GUI/script.js"
html_filename = f"https://raw.githubusercontent.com/ndvp39/CloudComputing-tirgul/main/Project/GUI/index.html"
switchToScreen(css_filename, html_filename, js_filename)

#========================#
#                        #
# ADMIN PASSWORD: 123456 #
#                        #
#========================#